# Training AID dataset by SRCNN

- reference code: https://github.com/worldstrat/worldstrat.git

In [ ]:
%load_ext autoreload
%autoreload 2
from src.train import *
import sys

In [ ]:
default_train_command = [
    # Batch size, gpus, limits
    "train.py",
    "--batch_size", "16", 
    "--accelerator", "gpu",
    "--devices", "1",
    "--max_steps", "10000",
    "--precision", "16",
    "--num_workers", "0",

    # Model/Hyperparameters
    "--model", "SRCNN", # must be uppercase
    "--w_mse", "0.3",
    "--w_mae", "0.4",
    "--w_ssim", "0.3",
    "--hidden_channels", "256",
    "--residual_layers", "8",
    "--padding_mode", "reflect",
    "--sr_kernel_size", "5",
    # "--use_dropout",
    # "--use_batchnorm",
    "--learning_rate", "1e-3",

    # Data
    "--root", "AID-dataset/",
    "--zoom_factor", "4", # 모델에 영향
    "--output_size", "600", "600",
    "--chip_size", "600", "600",
    "--chip_stride", "600", "600",
    "--randomly_rotate_and_flip_images",
    "--shuffle",
    "--subset_train", "0.6",

    # logging
    "--use_wandb",
    # "--benchmark_logging", # require enable wandb
    "--upload_checkpoint",

    # early stopping
    "--early_stop",
    "--early_stop_patience", "8",
    "--early_stop_min_delta", "0.001",
]

def run_training_command(training_command, running_on_windows=True):
    sys.argv = training_command
    if running_on_windows:
        sys.argv += ["--num_workers", "0"] # 윈도우에서는 멀티프로세싱 비효율(than linux)
    cli_main()

### Training a single model

In [ ]:
run_training_command(default_train_command, running_on_windows=False)